# 🚑 Kvasir-SEG U-Net Demo

This notebook demonstrates how to load and run inference using a pretrained **U-Net** model on the **Kvasir-SEG** dataset.

In [ ]:
# ✅ Install dependencies
!pip install torch torchvision albumentations opencv-python matplotlib tqdm scikit-learn

In [ ]:
# 📁 Clone GitHub Repo
!git clone https://github.com/GaurangTari4/KvasirSeg-Unet.git
%cd KvasirSeg-Unet

In [ ]:
# 📦 Download Dataset
from dataset.download import download_and_extract_kvasir
download_and_extract_kvasir("./dataset")

In [ ]:
# 🔽 Download pretrained model
!gdown --id 1J0TW0M7q_BW0M3Wv7NSnNdrwDuL9B8kN -O models/unet_kvasirseg.pth

In [ ]:
# 📊 Import required modules
import torch
from dataset.loader import get_dataloader
from model.unet import UNet
from utils.visualize import plot_prediction

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load validation data
val_loader = get_dataloader("./dataset", batch_size=4, split='val')

# Load model
model = UNet(in_channels=3, out_channels=1)
model.load_state_dict(torch.load("models/unet_kvasirseg.pth", map_location=device))
model = model.to(device)
model.eval()

In [ ]:
# 🔍 Run Inference on a few samples
import torch

with torch.no_grad():
    inputs, masks = next(iter(val_loader))
    inputs, masks = inputs.to(device), masks.to(device)
    preds = model(inputs)
    plot_prediction(inputs[:4].cpu(), masks[:4].cpu(), preds[:4].cpu())